In [1]:
import os
import xarray as xr
import numpy as np
from scipy.ndimage import gaussian_filter
from glob import glob
from tqdm import tqdm

In [2]:
experiment_name = "INALT60.L120-KRS0020"  
data_resolution = "1d" 

datapath = f'/gxfs_work/geomar/smomw523/smoothed_data/{experiment_name}/'
if experiment_name.startswith("INALT60"):
    prefix = "2_"
elif experiment_name.startswith("INALT20"):
    prefix = "1_"
meshpath = [datapath + f'{prefix}{experiment_name}_mesh_mask.nc']

output_path = f"/gxfs_work/geomar/smomw523/smoothed_data/{experiment_name}/"

os.makedirs(output_path, exist_ok=True)

In [3]:
datestart, dateend = "2012-04-10", "2012-05-04"
periods = [
    #(20120101, 20120125), (20120126, 20120219), (20120220, 20120315),
    #(20120316, 20120409), (20120410, 20120504), (20120505, 20120529),
    #(20120530, 20120628), (20121027, 20121120), (20121121, 20121215),
    #(20120629, 20120728), (20120729, 20120827), (20120828, 20120926), (20120927, 20121026),
]

In [4]:
sigma = 3 #9 INALT 60, 3 for INALT20, corresponds to 66.6km  

In [5]:
grid_vars = {
        "T": "sossheig",
        "U": "vozocrtx",
        "V": "vomecrty"
    }

#for (datestart, dateend) in periods:
print(f"Starting: {datestart}-{dateend} ...")
    
filename_pattern = f'{prefix}{experiment_name}_{data_resolution}_{datestart.replace("-", "")}_{dateend.replace("-", "")}_grid_[TUV].nc' #TUV
data_in = sorted(glob(datapath + filename_pattern))
    
for file in tqdm(data_in):
    grid_key = file.split("_grid_")[-1][0]
    var_name = grid_vars.get(grid_key)

    new_filename = os.path.basename(file).replace("_grid_", f"_smoothed_{sigma}_grid_")
    new_filepath = os.path.join(output_path, new_filename)

    if os.path.exists(new_filepath):
        print(f"Skipping {new_filename} (already exists)")
        continue

    ds = xr.open_dataset(file)

    if var_name in ds:
        dims = ds[var_name].dims
        sigma_tuple = (0, 0, sigma, sigma) if any(d in dims for d in ["depthu", "depthv"]) else (0, sigma, sigma)
            
        smoothed_var = gaussian_filter(ds[var_name], sigma=sigma_tuple)
        ds[var_name] = (dims, smoothed_var)  

        ds.to_netcdf(new_filepath)

    ds.close()

Starting: 2012-04-10-2012-05-04 ...


100%|██████████| 3/3 [00:00<00:00, 1369.94it/s]

Skipping 2_INALT60.L120-KRS0020_1d_20120410_20120504_smoothed_3_grid_T.nc (already exists)
Skipping 2_INALT60.L120-KRS0020_1d_20120410_20120504_smoothed_3_grid_U.nc (already exists)
Skipping 2_INALT60.L120-KRS0020_1d_20120410_20120504_smoothed_3_grid_V.nc (already exists)
